# Importando as bibliotecas

In [2]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from collections import Counter
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

# Dataset

In [3]:
procData = ProcessarDados("../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv")

# Funções

In [4]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada
def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_f1 = []
    resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        # divisão os dados 
        X_train, X_test, y_train, y_test = procData.holdout(0.2)
        #print(Counter(y_test))

        # realizando o grid search para encontrar a melhor combinação entre o C, gamma e Kernel, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        clf = svm.SVC(random_state = None)
        grid_svm = GridSearchCV(clf, param_grid, cv=k_folds, scoring='accuracy', verbose=0)
        grid_svm.fit(X_train, y_train)

        # Treinando do modelo com os melhores parametros encontrados
        C_best = grid_svm.best_estimator_.C
        gamma_best = grid_svm.best_estimator_.gamma
        kernel_best = grid_svm.best_estimator_.kernel

        SVM = svm.SVC(random_state = None, C = C_best, gamma = gamma_best, kernel = kernel_best)
        SVM.fit(X_train, y_train)

        #testando o modelo
        y_pred = SVM.predict(X_test)
        if exibir_matriz_confusao:
            print(confusion_matrix(y_test, y_pred))

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        f1_score = 2 * (precision * recall) / (precision + recall)

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_f1.append(f1_score)

        best_parametros = "C: "+ str(C_best)+ " | Gamma: "+ str(gamma_best)+ " | Kernel: "+ str(kernel_best);
        resultados_parametros.append(best_parametros)


        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("F1-Score:",f1_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['f1'] = resultados_f1
    
    return resultados_gerais

In [5]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\t F1-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f1'], axis=0), np.std(dict_metricas['f1'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)

# Definindo os parâmetros

In [10]:
c_range = [10, 20, 30, 40, 50]
gammas = [0.01, 0.02, 0.03, 0.04, 0.05]
kernels = ["rbf"]
param_grid = {'C': c_range, 'gamma': gammas, 'kernel': kernels}

epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True
salvarResultados = True
rodadas=False

# Treinando e obtendo as métricas do modelo

In [11]:
# treinando o modelo
dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('SVM',dict_metricas, rodadas, salvarResultados)

[[29  5]
 [ 2 84]]
Rodada: # 0
C: 20 | Gamma: 0.04 | Kernel: rbf
Accuracy: 0.9416666666666667
Precision: 0.9438202247191011
Recall: 0.9767441860465116
F1-Score: 0.9599999999999999


[[23  8]
 [ 5 84]]
Rodada: # 1
C: 10 | Gamma: 0.02 | Kernel: rbf
Accuracy: 0.8916666666666667
Precision: 0.9130434782608695
Recall: 0.9438202247191011
F1-Score: 0.9281767955801105


[[22 11]
 [ 4 83]]
Rodada: # 2
C: 20 | Gamma: 0.05 | Kernel: rbf
Accuracy: 0.875
Precision: 0.8829787234042553
Recall: 0.9540229885057471
F1-Score: 0.9171270718232043


[[22  8]
 [ 2 88]]
Rodada: # 3
C: 40 | Gamma: 0.04 | Kernel: rbf
Accuracy: 0.9166666666666666
Precision: 0.9166666666666666
Recall: 0.9777777777777777
F1-Score: 0.946236559139785


[[17  8]
 [ 2 93]]
Rodada: # 4
C: 40 | Gamma: 0.05 | Kernel: rbf
Accuracy: 0.9166666666666666
Precision: 0.9207920792079208
Recall: 0.9789473684210527
F1-Score: 0.9489795918367347


[[20 12]
 [ 3 85]]
Rodada: # 5
C: 20 | Gamma: 0.05 | Kernel: rbf
Accuracy: 0.875
Precision: 0.8762886597

[[26  5]
 [ 1 88]]
Rodada: # 47
C: 10 | Gamma: 0.04 | Kernel: rbf
Accuracy: 0.95
Precision: 0.946236559139785
Recall: 0.9887640449438202
F1-Score: 0.967032967032967


[[20  8]
 [ 4 88]]
Rodada: # 48
C: 50 | Gamma: 0.01 | Kernel: rbf
Accuracy: 0.9
Precision: 0.9166666666666666
Recall: 0.9565217391304348
F1-Score: 0.9361702127659574


[[14 12]
 [ 4 90]]
Rodada: # 49
C: 20 | Gamma: 0.05 | Kernel: rbf
Accuracy: 0.8666666666666667
Precision: 0.8823529411764706
Recall: 0.9574468085106383
F1-Score: 0.9183673469387754


============================================== SVM =================================================
=================================== TABELA DE MÉTRICAS DO MODELO ===================================
	 Accuracy 	|	 Precision 	|	 Recall 	|	 F1-Score
      0.90 +- 0.03             0.91 +- 0.03          0.96 +- 0.02             0.94 +- 0.02
